# How to redispatch a workflow

The Covalent redispatch command allows re-executing a workflow with:

* New input parameters
* Replaced/updated task definitions for a collection of tasks

Furthermore, there is the option to reuse the previous results whenever possible.

First, construct a workflow.

In [1]:
import covalent as ct


In [2]:
@ct.electron
def task_1(a, b):
    return a + b


@ct.electron
def task_2(a, b):
    return a / b


@ct.electron
def task_2_redefined(a, b):
    return a * b


@ct.lattice
def workflow(a, b):
    res_1 = task_1(a, b)
    res_2 = task_2(res_1, b)
    return res_2


Next the workflow is dispatched using the some input parameters.

In [3]:
dispatch_id = ct.dispatch(workflow)(1, 2)
print(f"Dispatch id: {dispatch_id}")
result = ct.get_result(dispatch_id, wait=True)
print(f"Workflow execution status: {result.status}")
print(f"Workflow output: {result.result}")


Dispatch id: b428aec1-a8fc-416a-a61f-07c071f68ab0
Workflow execution status: COMPLETED
Workflow output: 1.5


### Case I: Redispatch workflow with redefined task and previous input parameters

In [4]:
redispatch_id = ct.redispatch(
    dispatch_id, 
    replace_electrons={"task_2": task_2_redefined}
)()
print(f"Redispatch id: {redispatch_id}")
result = ct.get_result(redispatch_id, wait=True)
print(f"Redispatched workflow execution status: {result.status}")
print(f"Redispatched workflow output: {result.result}")


Redispatch id: 020b8adf-6baa-40d8-8650-af18ea6910a1
Redispatched workflow execution status: COMPLETED
Redispatched workflow output: 6


.. tip:: By leaving the input parameters empty, the redispatch function will automatically dispatch the workflow with the previous input parameters.

### Case II: Reuse previous results

The workflow can also be re-executed so that previously computed results are reused as much as possible. 

In [5]:
redispatch_id = ct.redispatch(
    dispatch_id, 
    replace_electrons={"task_2": task_2_redefined}, 
    reuse_previous_results=True
)()
print(f"Redispatch id: {redispatch_id}")
result = ct.get_result(redispatch_id, wait=True)
print(f"Redispatched workflow execution status: {result.status}")
print(f"Redispatched workflow output: {result.result}")


Redispatch id: c98ba205-0bd2-4ec7-82ee-4ceaa8a2ad92
Redispatched workflow execution status: COMPLETED
Redispatched workflow output: 6


In [6]:
# This value should be True if previous results were reused.

print(result.get_node_result(0)["start_time"] == result.get_node_result(0)["end_time"])


True


.. note:: By default, `reuse_previous_results` is set to `False`. In order to override this, the parameter needs to be explicitly set to `True`.

.. warning:: When stochasticity is involved in the workflow, `reuse_previous_results` should not be set to `True`.

### Case III: Redispatch with updated input parameters

In order to evaluate the workflow with new input parameters, simply pass them as one would in the `ct.dispatch` command.

In [7]:
redispatch_id = ct.redispatch(
    dispatch_id, 
    replace_electrons={"task_2": task_2_redefined}, 
    reuse_previous_results=True
)(1, 4)
print(f"Redispatch id: {redispatch_id}")
result = ct.get_result(redispatch_id, wait=True)
print(f"Redispatched workflow execution status: {result.status}")
print(f"Redispatched workflow output: {result.result}")


Redispatch id: 712c9c45-8026-48a1-98b2-242ae15a683d
Redispatched workflow execution status: COMPLETED
Redispatched workflow output: 20
